In [1]:
import numpy as np 
import keras
import pickle
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.models import Model
plt.ion()

/home/cftang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data.pickle', 'rb') as f:
    data_overall = pickle.load(f)

In [3]:
len(data_overall)

7701

In [4]:
random.shuffle(data_overall)

In [5]:
data = []
label = []

In [6]:
for d in data_overall:
    data.append(d[1])
    label.append(d[0])

data = np.array(data)
label = np.array(label)
del data_overall

In [7]:
data = data / 255
label = to_categorical(label)

In [8]:
data.shape

(7701, 224, 224, 3)

In [9]:
label.shape

(7701, 152)

In [10]:
split_number = 5000

In [11]:
train_X, train_y, test_X, test_y = data[:split_number,:, :, :], label[:split_number, :], data[split_number:, :, :, :], label[split_number:, :] 

In [12]:
print(train_X.shape)
print(train_y.shape)

(5000, 224, 224, 3)
(5000, 152)


In [13]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape = (224, 224, 3))

In [14]:
x = vgg16.output
x = Flatten()(x)
predict = Dense(152, activation='relu')(x)

In [15]:
model = Model(inputs=vgg16.input, outputs=predict)

for layer in vgg16.layers:
    layer.trainable = False

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [20]:
# train_X, train_y, test_X, test_y
batch_size = 8
epochs = 5

In [ ]:
model.fit(train_X, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_X, test_y))

Train on 5000 samples, validate on 2701 samples
Epoch 1/5
5000/5000 [==============================] - 121s 24ms/step - loss: nan - val_loss: nan
Epoch 2/5
5000/5000 [==============================] - 118s 24ms/step - loss: nan - val_loss: nan
Epoch 3/5
4992/5000 [============================>.] - ETA: 0s - loss: nan

In [ ]:
# class DataGenerator(keras.utils.Sequence):

#     def __init__(self, datas, labels, batch_size=1, shuffle=True):
#         self.batch_size = batch_size
#         self.datas = datas
#         self.labels = labels
#         self.indexes = np.arange(len(self.datas))
#         self.shuffle = shuffle

#     def __len__(self):
#         #计算每一个epoch的迭代次数
#         return math.ceil(len(self.datas) / float(self.batch_size))

#     def __getitem__(self, index):
#         #生成每个batch数据，这里就根据自己对数据的读取方式进行发挥了
#         # 生成batch_size个索引
#         batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
#         # 根据索引获取datas集合中的数据
#         batch_datas = [(self.labels[k], self.datas[k]) for k in batch_indexs]

#         # 生成数据
#         X, y = self.data_generation(batch_datas)

#         return X, y

#     def on_epoch_end(self):
#         #在每一次epoch结束是否需要进行一次随机，重新随机一下index
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def data_generation(self, batch_datas):
#         images = []
#         labels = []

#         # 生成数据
#         for data in batch_datas:
#             images.append(data[1])
#             labels.append(data[0])
            
#         return np.array(images), np.array(labels)


In [ ]:
# training_generator = DataGenerator(train_datas)